In [1]:
import pandas as pd
import numpy as np

### Import predicted data.

In [2]:
close_predict = pd.read_csv("close_predict.csv")
close_predict['date'] = pd.to_datetime(close_predict['Date'])
close_predict.set_index("Date", inplace=True)
close_predict = close_predict.iloc[:,:-1]
close_predict

,A,ALLE,AMGN,BIIB,BMY,CAT,ATVI,BDX,CHRW,CHD
Date,,,,,,,,,,
2022-08-29,131.380718,98.147062,241.181227,198.666519,66.405227,192.663060,78.987785,255.730668,116.043514,85.648039
2022-08-30,130.798271,98.679623,240.662175,198.064133,67.219551,188.031071,78.992458,254.946210,115.926283,85.300254
2022-08-31,130.957069,96.844718,240.577061,197.168918,67.592055,186.060007,78.817959,254.642618,115.302283,84.281325
2022-09-01,130.513179,96.534220,241.777353,197.033037,67.973690,184.579792,78.766513,254.970868,114.671812,84.295453
2022-09-02,130.125310,96.291704,244.860897,199.316352,69.667009,183.629147,77.793111,255.481074,115.189133,85.241380
...,...,...,...,...,...,...,...,...,...,...
2022-11-18,144.987439,110.500820,285.751894,298.971414,77.343919,229.976672,73.895758,221.425067,94.439858,75.602828
2022-11-21,145.022498,111.488692,287.462013,299.340821,77.276325,229.899213,74.373846,232.159791,95.423459,76.939432
2022-11-22,151.478534,109.401081,287.540644,301.938714,78.343670,230.920659,75.430741,233.170422,96.094731,77.545441


In [3]:
close_true = pd.read_csv("close_true.csv")
close_true['date'] = pd.to_datetime(close_true['Date'])
close_true.set_index("Date", inplace=True)
close_true = close_true.iloc[:,:-1]
close_true

,A,ALLE,AMGN,BIIB,BMY,CAT,ATVI,BDX,CHRW,CHD
Date,,,,,,,,,,
2022-08-29,128.110001,96.720001,239.380005,197.110001,66.750000,191.779999,78.750000,254.440002,115.029999,85.320000
2022-08-30,128.279999,96.690002,239.119995,196.080002,67.199997,186.940002,78.620003,253.169998,114.489998,84.720001
2022-08-31,128.250000,95.099998,240.300003,195.380005,67.410004,184.710007,78.489998,252.419998,114.150002,83.709999
2022-09-01,128.929993,95.730003,245.500000,199.789993,69.089996,182.440002,78.550003,255.250000,114.879997,84.860001
2022-09-02,128.009995,94.889999,242.369995,196.029999,68.610001,180.830002,77.529999,252.839996,115.360001,84.180000
...,...,...,...,...,...,...,...,...,...,...
2022-11-18,146.190002,112.959999,287.290009,302.890015,77.449997,231.429993,74.099998,225.589996,97.089996,76.419998
2022-11-21,145.139999,112.660004,288.160004,301.630005,78.959999,232.169998,74.800003,234.690002,97.570000,77.599998
2022-11-22,156.860001,113.940002,287.049988,306.720001,78.860001,237.259995,75.879997,235.610001,97.650002,78.089996


In [4]:
def get_close_predict(stock_name):
    return close_predict[stock_name]

In [5]:
def get_close_true(stock_name):
    return close_true[stock_name]

### Compute the nature of change

In [6]:
def delta(stock_name):
    close_predict=get_close_predict(stock_name)
    n=len(close_predict)
    delt=[0]*n
    for i in range(1,n):
        if close_predict[i]-close_predict[i-1]>0:
            delt[i]=1
        if close_predict[i]-close_predict[i-1]<0:
            delt[i]=-1
        else:
            delt[i]=0
    return delt

### Compute the curvature of the forecast

In [7]:
def Delta(stock_name):
    close_predict=get_close_predict(stock_name)
    n=len(close_predict)
    delt=delta(stock_name)
    Delt=[0]*(n-1)
    for i in range(1,n-1):
        Delt[i]=delt[i+1]-delt[i]
    return Delt

### Calculate the profit with initial values

In [8]:
def calculate_profit(stock_name,budget=10000,sell_num=1,buy_num=1,stock_num=0):
    close_predict=get_close_predict(stock_name)
    close_true=get_close_true(stock_name)
    close_predict=get_close_predict(stock_name)
    n=len(close_predict)
    delt=delta(stock_name)
    Delt=Delta(stock_name)
    initial_budget=budget
    for i in range(0,n-2):
        if Delt[i]==1 and stock_num>sell_num:
            budget=budget+close_true[i]*sell_num
            stock_num=stock_num-sell_num
        if Delt[i]==-1 and budget>close_true[i]*buy_num:
            stock_num=stock_num+buy_num
            budget=budget-close_true[i]*buy_num
        else:
            budget=budget
            stock_num=stock_num
            profit=budget+stock_num*close_true[n-1]-initial_budget
    return profit

### Find the best sell number and buy number for each time

In [13]:
def find_best(stock_name,budget):
    profit=np.zeros((100,100))
    for k in range(100):
        for l in range(100):
            profit[k,l]=calculate_profit(stock_name,budget,sell_num=k,buy_num=l)
    best_profit=np.max(profit)
    best_sell_num,best_buy_num=np.where(profit==best_profit)
    print(f'For {stock_name}, the best sell number is {best_sell_num[0]}, the best buy number is {best_buy_num[0]} and the best profit is {best_profit}')
    return best_sell_num,best_buy_num,best_profit

In [23]:
stocks=['A','ALLE','AMGN','BIIB','BMY','CAT']
budgets=[3300,1200,3000,700,1200,700]
sum(budgets)

10100

In [19]:
best_profit=np.ones(6)
best_profit

array([1., 1., 1., 1., 1., 1.])

In [20]:
for i in range(len(stocks)):
    best_profit[i]=find_best(stocks[i],budget=budgets[i])[2]

For A, the best sell number is 0, the best buy number is 40 and the best profit is 1368.8003540039072
For ALLE, the best sell number is 0, the best buy number is 13 and the best profit is 301.8600158691406
For AMGN, the best sell number is 0, the best buy number is 13 and the best profit is 688.7399444580074
For BIIB, the best sell number is 0, the best buy number is 3 and the best profit is 327.3599853515625
For BMY, the best sell number is 0, the best buy number is 17 and the best profit is 180.70995330810547
For CAT, the best sell number is 0, the best buy number is 4 and the best profit is 279.20001220703125


In [21]:
best=sum(best_profit)
print(f'our best total profit is {best}')

our best total profit is 3146.6702651977544
